## Imports

In [3]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [5]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
# week_ending = format_date_with_ordinal(today)
# week_starting = format_date_with_ordinal(one_week_ago)

week_ending = 'Dec. 9th'
week_starting = 'Dec. 2nd'

print(week_starting, week_ending)

Dec. 2nd Dec. 9th


In [14]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [15]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [16]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [17]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [22]:
len(df)

112

In [20]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [21]:

# Define the date range
start_date = '2024-12-02'
end_date = '2024-12-09'

# Filter the DataFrame between two dates (inclusive)
df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [23]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
33,PAST SALE,2024-12-09,Condo/Co-op,7161 Promenade Dr #502,Boca Raton,FL,33433.0,565000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7161-Prom...,MARMLS,A11681939,N,Y,26.335179,-80.156517
98,PAST SALE,2024-12-09,Condo/Co-op,1038 Cornwall B,Boca Raton,FL,33434.0,145000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1038-Corn...,Beaches MLS,RX-10998207,N,Y,26.386660,-80.180173
123,PAST SALE,2024-12-09,Condo/Co-op,4735 Lucerne Lakes Blvd E #209,Lake Worth,FL,33467.0,160000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/4735-Luce...,Beaches MLS,RX-10986824,N,Y,26.603860,-80.161952
138,PAST SALE,2024-12-09,Condo/Co-op,100 Ocean Trail Way #1305,Jupiter,FL,33477.0,745000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/100-Ocean-Tr...,Beaches MLS,RX-11022203,N,Y,26.935321,-80.070170
194,PAST SALE,2024-12-09,Condo/Co-op,21214 Lago Cir Unit G,Boca Raton,FL,33433.0,355000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/21214-Lag...,Beaches MLS,RX-11002695,N,Y,26.357939,-80.177547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,PAST SALE,2024-12-02,Condo/Co-op,5217 Brisata Cir Unit E,Boynton Beach,FL,33437.0,280000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/5217-B...,Beaches MLS,RX-11014732,N,Y,26.528654,-80.125220
129,PAST SALE,2024-12-02,Condo/Co-op,4200 N Ocean Dr Unit 2-204,Singer Island,FL,33404.0,450000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4200-N...,Beaches MLS,RX-11024536,N,Y,26.797038,-80.032240
144,PAST SALE,2024-12-02,Condo/Co-op,1099 S Ocean Blvd S #305,Boca Raton,FL,33432.0,555750.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1099-S-Oc...,Beaches MLS,RX-11012275,N,Y,26.336492,-80.074089
152,PAST SALE,2024-12-02,Condo/Co-op,3636 Whitehall Dr #402,West Palm Beach,FL,33401.0,235000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3636...,Beaches MLS,RX-11020639,N,Y,26.747287,-80.087226


In [24]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [25]:
df_top_ten = df.head(10)

## Collect Agent Information

In [26]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [27]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [28]:
response_df = pd.DataFrame(response_list)

In [29]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [30]:
df_top_ten = merged_df

## Current Week's Values

In [31]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

112


In [32]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$434,716


In [33]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$297


In [34]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$48,688,200


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [35]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
53
Input Previous Week Condo Average Sales Price:
334751
Input Previous Week Condo Average PSF:
276
Input Previous Week Condo Sales total (ex: 198_000_000)
17_700_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [36]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [37]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [38]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [39]:
pd.set_option('display.max_columns',None)

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange


In [41]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange
1,PAST SALE,2024-12-03,Condo/Co-op,200 SE Mizner Blvd #609,Boca Raton,FL,33432.0,3600000.0,2.0,3.5,Alina Boca Raton Condo,2897.0,NaN,2021.0,NaN,1243.0,3801.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11010104,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,Edward Pitlake,Douglas Elliman,Aaron Buchbinder,Compass Florida LLC,Aaron Buchbinder,Compass Florida LLC,2,blue
2,PAST SALE,2024-12-03,Condo/Co-op,475 E Royal Palm Rd #406,Boca Raton,FL,33432.0,2800000.0,3.0,3.5,Royal Palm Residences,2966.0,50190.0,2024.0,NaN,944.0,2789.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Beaches MLS,RX-10972164,N,Y,26.349927,-80.078241,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Rochelle LeCavalier,Douglas Elliman,None,None,Warren Drucker,Keller Williams Realty Boca Raton,None,None,3,blue
3,PAST SALE,2024-12-04,Condo/Co-op,3100 N Ocean Dr #906,Singer Island,FL,33404.0,1700000.0,2.0,2.0,Amrit Ocean Resort & Resi,1620.0,NaN,2023.0,NaN,1049.0,1800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10415336,N,Y,26.788079,-80.034376,https://www.redfin.com/FL/Riviera-Beach/3100-N...,None,None,None,None,None,None,None,None,4,blue
4,PAST SALE,2024-12-02,Condo/Co-op,2000 S Ocean Blvd Unit 15b,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Whitehall,1236.0,NaN,1971.0,NaN,1011.0,1464.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024358,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,5,blue
5,PAST SALE,2024-12-04,Condo/Co-op,4740 S Ocean Blvd #816,Highland Beach,FL,33487.0,1085000.0,3.0,2.5,Braemar Isle Condo,2200.0,NaN,1982.0,NaN,493.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11005627,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Davina Cook,One Sotheby's International Realty,None,None,6,blue
6,PAST SALE,2024-12-06,Condo/Co-op,700 E Boynton Beach Blvd #1002,Boynton Beach,FL,33435.0,1000000.0,3.0,2.0,Marina Village AT Boynton Beach Condo,1646.0,NaN,2006.0,NaN,608.0,1466.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Beaches MLS,RX-11005280,N,Y,26.528796,-80.055894,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Lisa Bailey Levy,EXP Realty LLC,None,None,Mary Ann Prince,South Florida Realty Group & Property Manageme...,None,None,7,blue
7,PAST SALE,2024-12-05,Condo/Co-op,4100 N Ocean Dr #1201,Singer Island,FL,33404.0,925000.0,2.0,3.5,Martinique 2 Condo,2011.0,NaN,1986.0,NaN,460.0,1711.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Beaches MLS,RX-11010371,N,Y,26.796399,-80.033363,https://www.redfin.com/FL/Riviera-Beach/4100-N...,Mariela Vega-Herklotz,Keller Williams Realty/P B,None,None,Maria Demps,Illustrated Properties LLC / S,None,None,8,blue
8,PAST SALE,2024-12-06,Condo/Co-

In [44]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Palm Beach County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [45]:
m.save('index.html')

## Data snagger

In [46]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [47]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [48]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_12092024


# CREATE TEMPLATE 

In [49]:
muni_set = set(df_top_ten['CITY'])

In [50]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [51]:
df_top_ten.reset_index(inplace=True,drop=True)

In [52]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [53]:
top_sale

'Beresford at 350 S Ocean Blvd Unit 5a in Boca Raton'

In [54]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [55]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [56]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 5a in Boca ...
1,PAST SALE,2024-12-03,Condo/Co-op,200 SE Mizner Blvd #609,Boca Raton,FL,33432.0,3600000.0,2.0,3.5,Alina Boca Raton Condo,2897.0,NaN,2021.0,NaN,1243.0,3801.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11010104,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,Edward Pitlake,Douglas Elliman,Aaron Buchbinder,Compass Florida LLC,Aaron Buchbinder,Compass Florida LLC,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2024-12-03,Condo/Co-op,475 E Royal Palm Rd #406,Boca Raton,FL,33432.0,2800000.0,3.0,3.5,Royal Palm Residences,2966.0,50190.0,2024.0,NaN,944.0,2789.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Beaches MLS,RX-10972164,N,Y,26.349927,-80.078241,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Rochelle LeCavalier,Douglas Elliman,None,None,Warren Drucker,Keller Williams Realty Boca Raton,None,None,3,blue,Royal Palm Residences at 475 E Royal Palm Rd #...
3,PAST SALE,2024-12-04,Condo/Co-op,3100 N Ocean Dr #906,Singer Island,FL,33404.0,1700000.0,2.0,2.0,Amrit Ocean Resort & Resi,1620.0,NaN,2023.0,NaN,1049.0,1800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10415336,N,Y,26.788079,-80.034376,https://www.redfin.com/FL/Riviera-Beach/3100-N...,None,None,None,None,None,None,None,None,4,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr #...
4,PAST SALE,2024-12-02,Condo/Co-op,2000 S Ocean Blvd Unit 15b,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Whitehall,1236.0,NaN,1971.0,NaN,1011.0,1464.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024358,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,5,blue,Whitehall at 2000 S Ocean Blvd Unit 15b in Boc...
5,PAST SALE,2024-12-04,Condo/Co-op,4740 S Ocean Blvd #816,Highland Beach,FL,33487.0,1085000.0,3.0,2.5,Braemar Isle Condo,2200.0,NaN,1982.0,NaN,493.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11005627,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Davina Cook,One Sotheby's International Realty,None,None,6,blue,Braemar Isle Condo at 4740 S Ocean Blvd #816 i...
6,PAST SALE,2024-12-06,Condo/Co-op,700 E Boynton Beach Blvd #1002,Boynton Beach,FL,33435.0,1000000.0,3.0,2.0,Marina Village AT Boynton Beach Condo,1646.0,NaN,2006.0,NaN,608.0,1466.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Beaches MLS,RX-11005280,N,Y,26.528796,-80.055894,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Lisa Bailey Levy,EXP Realty LLC,None,None,Mary Ann Prince,South Florida Realty Group & Property Manageme...,None,None,7,blue,Marina Village AT Boynton Beach Condo at 700 E...
7,PAST SALE,2024-12-05,Condo/Co-op,4100 N Ocean Dr #1201,Singer Island,FL,33404.0,925000.0,2.0,3.5,Martini

In [57]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Beresford closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $825,000M to $3,650,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Highland Beach, Singer Island, West Palm Beach, Boynton Beach, Boca Raton, Palm Beach Gardens

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 112 condo sales totaling $48,688,200 million from Dec. 2nd to Dec. 9th. The previous week, brokers closed 53 condo sales totaling $17,700,000.

Last week’s units sold for an average of $434,716

In [58]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [59]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [60]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 5a in Boca ...
1,PAST SALE,2024-12-03,Condo/Co-op,200 SE Mizner Blvd #609,Boca Raton,FL,33432.0,3600000.0,2.0,3.5,Alina Boca Raton Condo,2897.0,NaN,2021.0,NaN,1243.0,3801.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11010104,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,Edward Pitlake,Douglas Elliman,Aaron Buchbinder,Compass Florida LLC,Aaron Buchbinder,Compass Florida LLC,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2024-12-03,Condo/Co-op,475 E Royal Palm Rd #406,Boca Raton,FL,33432.0,2800000.0,3.0,3.5,Royal Palm Residences,2966.0,50190.0,2024.0,NaN,944.0,2789.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Beaches MLS,RX-10972164,N,Y,26.349927,-80.078241,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Rochelle LeCavalier,Douglas Elliman,None,None,Warren Drucker,Keller Williams Realty Boca Raton,None,None,3,blue,Royal Palm Residences at 475 E Royal Palm Rd #...
3,PAST SALE,2024-12-04,Condo/Co-op,3100 N Ocean Dr #906,Singer Island,FL,33404.0,1700000.0,2.0,2.0,Amrit Ocean Resort & Resi,1620.0,NaN,2023.0,NaN,1049.0,1800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10415336,N,Y,26.788079,-80.034376,https://www.redfin.com/FL/Riviera-Beach/3100-N...,None,None,None,None,None,None,None,None,4,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr #...
4,PAST SALE,2024-12-02,Condo/Co-op,2000 S Ocean Blvd Unit 15b,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Whitehall,1236.0,NaN,1971.0,NaN,1011.0,1464.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024358,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,5,blue,Whitehall at 2000 S Ocean Blvd Unit 15b in Boc...
5,PAST SALE,2024-12-04,Condo/Co-op,4740 S Ocean Blvd #816,Highland Beach,FL,33487.0,1085000.0,3.0,2.5,Braemar Isle Condo,2200.0,NaN,1982.0,NaN,493.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11005627,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Davina Cook,One Sotheby's International Realty,None,None,6,blue,Braemar Isle Condo at 4740 S Ocean Blvd #816 i...
6,PAST SALE,2024-12-06,Condo/Co-op,700 E Boynton Beach Blvd #1002,Boynton Beach,FL,33435.0,1000000.0,3.0,2.0,Marina Village AT Boynton Beach Condo,1646.0,NaN,2006.0,NaN,608.0,1466.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Beaches MLS,RX-11005280,N,Y,26.528796,-80.055894,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Lisa Bailey Levy,EXP Realty LLC,None,None,Mary Ann Prince,South Florida Realty Group & Property Manageme...,None,None,7,blue,Marina Village AT Boynton Beach Condo at 700 E...
7,PAST SALE,2024-12-05,Condo/Co-op,4100 N Ocean Dr #1201,Singer Island,FL,33404.0,925000.0,2.0,3.5,Martini

In [62]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Beresford closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $825,000M to $3,650,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Highland Beach, Singer Island, West Palm Beach, Boynton Beach, Boca Raton, Palm Beach Gardens

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 112 condo sales totaling $48,688,200 million from Dec. 2nd to Dec. 9th. The previous week, brokers closed 53 condo sales totaling $17,700,000.

Last week’s units sold for an average of $434,716

In [63]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/350-S-Ocean-Blvd-33432/unit-5-A/home/182981890


In [71]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/200-SE-Mizner-Blvd-33432/unit-609/home/175171027


In [65]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/West-Palm-Beach/1900-Consulate-Pl-33401/unit-304/home/42520987


In [66]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Palm-Beach-Gardens/115-Palm-Point-Cir-33418/unit-D/home/42547877


In [67]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/350-S-Ocean-Blvd-33432/unit-5-A/home/182981890


In [68]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/115-Palm-Point-Cir-33418/unit-D/home/42547877


In [69]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,350 S Ocean Blvd Unit 5a,Boca Raton,FL,33432.0,3650000.0,3.0,3.0,Beresford,2523.0,NaN,1977.0,NaN,1447.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11012591,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Botta,One Sotheby's International Realty,None,None,Michael Botta,One Sotheby's International Realty,None,None,1,orange,Beresford at 350 S Ocean Blvd Unit 5a in Boca ...
1,PAST SALE,2024-12-03,Condo/Co-op,200 SE Mizner Blvd #609,Boca Raton,FL,33432.0,3600000.0,2.0,3.5,Alina Boca Raton Condo,2897.0,NaN,2021.0,NaN,1243.0,3801.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11010104,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,Edward Pitlake,Douglas Elliman,Aaron Buchbinder,Compass Florida LLC,Aaron Buchbinder,Compass Florida LLC,2,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
2,PAST SALE,2024-12-03,Condo/Co-op,475 E Royal Palm Rd #406,Boca Raton,FL,33432.0,2800000.0,3.0,3.5,Royal Palm Residences,2966.0,50190.0,2024.0,NaN,944.0,2789.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Beaches MLS,RX-10972164,N,Y,26.349927,-80.078241,https://www.redfin.com/FL/Boca-Raton/475-E-Roy...,Rochelle LeCavalier,Douglas Elliman,None,None,Warren Drucker,Keller Williams Realty Boca Raton,None,None,3,blue,Royal Palm Residences at 475 E Royal Palm Rd #...
3,PAST SALE,2024-12-04,Condo/Co-op,3100 N Ocean Dr #906,Singer Island,FL,33404.0,1700000.0,2.0,2.0,Amrit Ocean Resort & Resi,1620.0,NaN,2023.0,NaN,1049.0,1800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,F10415336,N,Y,26.788079,-80.034376,https://www.redfin.com/FL/Riviera-Beach/3100-N...,None,None,None,None,None,None,None,None,4,blue,Amrit Ocean Resort & Resi at 3100 N Ocean Dr #...
4,PAST SALE,2024-12-02,Condo/Co-op,2000 S Ocean Blvd Unit 15b,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Whitehall,1236.0,NaN,1971.0,NaN,1011.0,1464.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11024358,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,5,blue,Whitehall at 2000 S Ocean Blvd Unit 15b in Boc...
5,PAST SALE,2024-12-04,Condo/Co-op,4740 S Ocean Blvd #816,Highland Beach,FL,33487.0,1085000.0,3.0,2.5,Braemar Isle Condo,2200.0,NaN,1982.0,NaN,493.0,2144.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11005627,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Davina Cook,One Sotheby's International Realty,None,None,6,blue,Braemar Isle Condo at 4740 S Ocean Blvd #816 i...
6,PAST SALE,2024-12-06,Condo/Co-op,700 E Boynton Beach Blvd #1002,Boynton Beach,FL,33435.0,1000000.0,3.0,2.0,Marina Village AT Boynton Beach Condo,1646.0,NaN,2006.0,NaN,608.0,1466.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Beaches MLS,RX-11005280,N,Y,26.528796,-80.055894,https://www.redfin.com/FL/Boynton-Beach/700-E-...,Lisa Bailey Levy,EXP Realty LLC,None,None,Mary Ann Prince,South Florida Realty Group & Property Manageme...,None,None,7,blue,Marina Village AT Boynton Beach Condo at 700 E...
7,PAST SALE,2024-12-05,Condo/Co-op,4100 N Ocean Dr #1201,Singer Island,FL,33404.0,925000.0,2.0,3.5,Martini

## Time on Market Calculator

In [72]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 5) ## List (Earlier) date
date2 = datetime(2024, 12, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

120


## Clean CSV for Datawrapper Chart

In [83]:
chart_df = df_top_ten

In [84]:
chart_df = chart_df.fillna(" ")

In [85]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [86]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [87]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [88]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [89]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [90]:
csv_date_string = today.strftime("%m_%d_%Y")

In [91]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [92]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-05,350 S Ocean Blvd Unit 5a in Boca Raton,"$3,650,000",3,3.0,"2,523",1977,"$1,447",Beaches MLS,RX-11012591,Michael Botta One Sotheby's International Real...,Michael Botta One Sotheby's International Real...
1,2024-12-03,200 SE Mizner Blvd #609 in Boca Raton,"$3,600,000",2,3.5,"2,897",2021,"$1,243",Beaches MLS,RX-11010104,Kathy Koch Pitlake Douglas Elliman Edward Pitl...,Aaron Buchbinder Compass Florida LLC Aaron Buc...
2,2024-12-03,475 E Royal Palm Rd #406 in Boca Raton,"$2,800,000",3,3.5,"2,966",2024,$944,Beaches MLS,RX-10972164,Rochelle LeCavalier Douglas Elliman,Warren Drucker Keller Williams Realty Boca Rat...
3,2024-12-04,3100 N Ocean Dr #906 in Singer Island,"$1,700,000",2,2.0,"1,620",2023,"$1,049",Beaches MLS,F10415336,,
4,2024-12-02,2000 S Ocean Blvd Unit 15b in Boca Raton,"$1,250,000",2,2.0,"1,236",1971,"$1,011",Beaches MLS,RX-11024358,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...
5,2024-12-04,4740 S Ocean Blvd #816 in Highland Beach,"$1,085,000",3,2.5,"2,200",1982,$493,Beaches MLS,RX-11005627,Michael Wells Premier Estate Properties Inc Sc...,Davina Cook One Sotheby's International Realty...
6,2024-12-06,700 E Boynton Beach Blvd #1002 in Boynton Beach,"$1,000,000",3,2.0,"1,646",2006,$608,Beaches MLS,RX-11005280,Lisa Bailey Levy EXP Realty LLC,Mary Ann Prince South Florida Realty Group & P...
7,2024-12-05,4100 N Ocean Dr #1201 in Singer Island,"$925,000",2,3.5,"2,011",1986,$460,Beaches MLS,RX-11010371,Mariela Vega-Herklotz Keller Williams Realty/P...,Maria Demps Illustrated Properties LLC / S
8,2024-12-06,115 Palm Point Cir Unit D in Palm Beach Gardens,"$825,000",3,3.0,"2,492",2000,$331,Beaches MLS,RX-11030119,Lee Weisberg Signature Best Florida Realty Jil...,"Andrew Leibowitz Leibowitz Realty Group, LLC./..."
9,2024-12-06,1900 Consulate Pl #304 in West Palm Beach,"$825,000",3,2.0,"2,274",1980,$363,Beaches MLS,RX-11004709,"Andrew Thomka-Gazdik Sotheby's Intl. Realty, I...","Suzanne Stoll Sotheby's Intl. Realty, Inc."
